## Import

In [1]:
import os
import cv2
from PIL import Image
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.optim.lr_scheduler import _LRScheduler

from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

from torchvision import models
from torchsummary import summary
import torch.nn.functional as F

# GPU 사용이 가능할 경우, GPU를 사용할 수 있게 함.'
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
device = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device)
print(device)

print(os.environ.get('CUDA_VISIBLE_DEVICES'))

/home/MMI24limbyungwan/.conda/envs/byungwan_resn/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda
2


## Utils

In [2]:
# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

# 클래스별 IoU를 계산하기 위한 함수
def calculate_iou_per_class(y_true, y_pred, class_id):
    intersection = np.sum((y_true == class_id) & (y_pred == class_id))
    union = np.sum((y_true == class_id) | (y_pred == class_id))
    iou = intersection / union if union > 0 else 0
    return iou

## Dataset, Data Loader

출력이미지 크기 키우기->ex) resnet 2048->1024->512->256 conv 256->512->1024->2048

In [3]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None, infer=False):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.infer = infer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        directory_path = "/mnt/nas27/Dataset/Samsung_DM"
        img_path = self.data.iloc[idx, 1]
        img_path = os.path.join(directory_path, img_path[2:])
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        if self.infer:
            if self.transform:
                image = self.transform(image=image)['image']
            return image
        
        mask_path = self.data.iloc[idx, 2]
        mask_path = os.path.join(directory_path, mask_path[2:])
        mask = cv2.imread(mask_path)
        #mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        mask[mask == 255] = 12 #배경을 픽셀값 12로 간주

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask
    
    # 이미지 전처리 클래스
class ImageTransform():
  """
  훈련, 검증 동작 다르게 설정
  이미지 크기 resize, 색상 표준화
  훈련시 RandomResizedCrop, RandomHorizontalFilp으로 데이터 확장
  """
  def __init__(self, resize, mean, std):
    self.data_transform = {
        'train' : transforms.Compose([
            #transforms.RandomResizedCrop(
            #    resize, scale = (0.5, 1.0)), # 데이터 확장
            #transforms.RandomHorizontalFlip(), # 데이터 확장
            transforms.ToTensor(), # Tensor로 변환
            transforms.Normalize(mean = mean, std = std) #표준화
        ]),
        'test': transforms.Compose([
            #transforms.Resize(resize), # Resize
            #transforms.CenterCrop(resize), # 중앙을 resize*resize로 crop
            transforms.ToTensor(), # Tensor로 변환
            transforms.Normalize(mean = mean, std = std) # 표준화
        ])

    }
  def __call__(self, img, phase = 'train'):
    """
    phase : 'train' or 'test'
    전처리 모드 지정
    """
    return self.data_transform[phase](img)


transform = A.Compose(
    [   
        #A.Resize(224, 224),
        A.Resize(128, 128),
        A.Normalize(),
        
        # 변형
        # A.VerticalFlip(p=0.5),
        # A.RandomRotate90(p=0.5),
        # A.HueSaturationValue(p=0.2),
        
        ToTensorV2()
    ]
)

Warmup

In [4]:
class WarmUpLR(_LRScheduler):
    def __init__(self, optimizer, total_iters, last_epoch=-1):
        self.total_iters = total_iters
        super(WarmUpLR, self).__init__(optimizer, last_epoch)

    def get_lr(self):
        return [base_lr * self.last_epoch / self.total_iters for base_lr in self.base_lrs]


## Define Model

In [5]:
#Unet의 기본이 되는 conv블럭
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu1 = nn.ReLU()
        
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)  # 여기서 in_channels는 out_channels와 동일해야 합니다.
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu2 = nn.ReLU()

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        return x

#인코더 블럭
class EncoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(EncoderBlock,self).__init__()
        self.convblock1 = ConvBlock(in_channels, out_channels)  # 첫 번째 ConvBlock의 in_channels는 입력 이미지의 채널 수와 일치해야 합니다.
        #self.convblock2 = ConvBlock(out_channels, out_channels)  # 두 번째 ConvBlock의 in_channels는 out_channels와 일치해야 합니다.
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self,x):
        x = self.convblock1(x)
        #x = self.convblock2(x)
        p = self.maxpool(x)
        return x , p
#디코더 블럭
#디코더는 업샘플링 이후 스킵연결과 붙어서 convblock을 통과해야함
#skip보다 작은 x x먼저 업샘플링 32 -> 64 , skip과 결합 6464 
class DecoderBlock(nn.Module):
    def __init__(self, channels):
        super(DecoderBlock,self).__init__()
        self.upsample = nn.ConvTranspose2d(channels*2, channels, kernel_size=4, stride=2, padding=1)#x 업샘플링
        self.convblock1 = ConvBlock(channels*2, channels)#차원감소
        #self.convblock2 = ConvBlock(channels, channels)
    def forward(self,x,skip):
        x = self.upsample(x)
        x = torch.cat([x, skip], dim=1)
        x = self.convblock1(x)
        #x = self.convblock2(x)
        return x
        
#Unet구조 middle의 xm값의 움직임에 주의
class Unet(nn.Module):
    def __init__(self,n_classes):
        super(Unet,self).__init__()
        self.encoder1 = EncoderBlock(3,64)
        self.encoder2 = EncoderBlock(64,128)
        self.encoder3 = EncoderBlock(128,256)
        self.encoder4 = EncoderBlock(256,512)
        
        self.middleconv = ConvBlock(512,1024)
        
        
        self.decoder4 = DecoderBlock(512)
        self.decoder3 = DecoderBlock(256)
        self.decoder2 = DecoderBlock(128)
        self.decoder1 = DecoderBlock(64)
        self.segmap = nn.Conv2d(64,n_classes, kernel_size=1)
        
        
    def forward(self,x):
        x1,p = self.encoder1(x)#3->64   #P:256,256 x1 :512,512
        x2,p = self.encoder2(p)#64->128 #P:128,128 x2:256,256
        x3,p = self.encoder3(p)#128->256#p:64,64 x3:128,128
        x4,p = self.encoder4(p)#256->512#p:32,32 x4:64,64
        
        xm = self.middleconv(p)#512->1024#32,32
        
        x = self.decoder4(xm,x4)#뉴런:1024->512->512 #출력tensor:64,64
        x = self.decoder3(x,x3)#뉴런:512->256->256 #출력tensor:128,128
        x = self.decoder2(x,x2)#뉴런:256->128->128 #출력tensor:256,256
        x = self.decoder1(x,x1)#뉴런:128->64->64 #출력tensor:512,512
        x = self.segmap(x)
        return x

## ResNet50

In [6]:
# import torch.nn as nn
# import torchvision.models as models

# class UNetWithResnet50Encoder(nn.Module):
#     def __init__(self, n_classes):
#         super().__init__()
        
#         # ResNet50 백본을 불러옵니다.
#         self.resnet = models.resnet50(pretrained=True)
        
#         # 인코더 부분
#         self.encoder0 = nn.Sequential(
#             self.resnet.conv1,
#             self.resnet.bn1,
#             self.resnet.relu,
#             self.resnet.maxpool
#         )
#         self.encoder1 = self.resnet.layer1
#         self.encoder2 = self.resnet.layer2
#         self.encoder3 = self.resnet.layer3
#         self.encoder4 = self.resnet.layer4
        
#         # 중간 컨볼루션
#         self.middle_conv = nn.Sequential(
#             nn.Conv2d(2048, 1024, kernel_size=3, padding=1),
#             nn.ReLU()
#         )
        
#         # 디코더 부분
#         self.decoder3 = nn.Sequential(
#             nn.ConvTranspose2d(1024+1024, 512, kernel_size=2, stride=2),
#             nn.ReLU()
#         )
#         self.decoder2 = nn.Sequential(
#             nn.ConvTranspose2d(512+512, 256, kernel_size=2, stride=2),
#             nn.ReLU()
#         )
#         # 디코더 부분
#         self.decoder1 = nn.Sequential(
#             nn.ConvTranspose2d(256+256, 128, kernel_size=2, stride=2),  # 출력 채널 수를 128로 변경
#             nn.ReLU()
#         )
#         self.decoder0 = nn.Sequential(
#             nn.ConvTranspose2d(128+64, n_classes, kernel_size=2, stride=2),  # 입력 채널 수를 192로 변경
#             nn.ReLU()
#         )
        
#     def forward(self, x):
#         # 인코더 부분
#         x0 = self.encoder0(x)
#         x1 = self.encoder1(x0)
#         x2 = self.encoder2(x1)
#         x3 = self.encoder3(x2)
#         x4 = self.encoder4(x3)
        
#         # 중간 컨볼루션 후 업샘플링
#         middle = self.middle_conv(x4)
#         middle_upsampled = nn.functional.interpolate(middle, scale_factor=2, mode='bilinear', align_corners=True)

#         # 디코더 부분 (skip connection 추가)
#         d3 = self.decoder3(torch.cat([middle_upsampled, x3], dim=1))
#         d2 = self.decoder2(torch.cat([d3, x2], dim=1))
#         d1 = self.decoder1(torch.cat([d2, x1], dim=1))
#         out = self.decoder0(torch.cat([d1, x0], dim=1))
        
#         return out


In [7]:
LR = 0.001
EP = 10
BATCH_SIZE = 64
ACCMULATION_STEP = 1 
N_CLASSES = 13 #IoU 점수측정하기 위한 클래스의 개수
WUP_ITERS = 10  # 웜업을 위한 반복 횟수
# model 초기화
#model = Unet_resnet18(n_classes = N_CLASSES).to(device)
#model = ResNet50(num_classes=N_CLASSES).to(device)
model = Unet(n_classes = N_CLASSES).to(device)

# loss function과 optimizer 정의

criterion =nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=LR)
optimizer.zero_grad() 
#optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=0.9, nesterov=True)
# Warmup을 위한 스케줄러 설정
scheduler_warmup = WarmUpLR(optimizer, WUP_ITERS)

dataset = CustomDataset(csv_file=os.path.join("/mnt/nas27/Dataset/Samsung_DM",'./train_source.csv'), transform=transform)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8)
valid_dataset = CustomDataset(csv_file=os.path.join("/mnt/nas27/Dataset/Samsung_DM",'./val_source.csv'), transform=transform)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8)

In [8]:
# import matplotlib.pyplot as plt

# # 데이터로더에서 배치 하나를 가져옵니다.
# dataiter = iter(dataloader)
# images, masks = next(dataiter)

# # 이미지를 시각화합니다.
# for i in range(images.size(0)):
#     image = images[i].permute(1, 2, 0).numpy()  # 이미지를 CHW에서 HWC로 변환
#     mask = masks[i].numpy()

#     plt.figure(figsize=(8, 8))
#     plt.subplot(1, 2, 1)
#     plt.imshow(image)
#     plt.title("Image")

#     plt.subplot(1, 2, 2)
#     plt.imshow(mask, cmap='gray')
#     plt.title("Mask")

#     plt.show()


In [9]:
# for images, masks in tqdm(dataloader):
#     a = images.float().to(device)
#     b = masks.long().to(device)
#     break

# # PyTorch Tensor를 NumPy 배열로 변환
# a_numpy = a[3].cpu().numpy()  # 첫 번째 이미지만 선택하거나 필요한 이미지를 선택하세요.
# a_numpy = np.transpose(a_numpy, (1, 2, 0))
# b_numpy = b[3].cpu().numpy()
# b_numpy = b_numpy*12

# # 이미지를 저장
# cv2.imwrite('image_source.png', a_numpy)  # 이미지 저장
# cv2.imwrite('image_mask.png', b_numpy)   # 마스크 저장

In [10]:

# for epoch in range(2):  # 5 에폭 동안 학습합니다.
          
#     # 클래스별 IoU를 누적할 리스트 초기화
#     train_class_ious = []
    
#     # 학습
#     model.train()
#     epoch_loss = 0
    
#     for images, masks in tqdm(dataloader):
#         images = images.float().to(device)
#         masks = masks.long().to(device)

#         optimizer.zero_grad()
#         outputs = model(images)
#         mask_spueeze = masks.squeeze(1)
#         loss = criterion(outputs, masks.squeeze(1))
#         loss.backward()
#         optimizer.step()

#         epoch_loss += loss.item()

## Model Train

In [11]:
import random
torch.cuda.empty_cache()

for epoch in range(EP):
    # 클래스별 IoU를 누적할 리스트 초기화
    train_class_ious = []
    # 학습
    model.train()
    epoch_loss = 0
    for images, masks in tqdm(dataloader):
        images = images.float().to(device)
        masks = masks.long().to(device)

        outputs = model(images)
        loss = criterion(outputs, masks.squeeze(1))
        loss.backward()
        if (epoch+1) % ACCMULATION_STEP == 0:
            optimizer.step()
            optimizer.zero_grad()
            # # Warmup 스케줄러 업데이트
            # if epoch < WUP_ITERS:
            #     scheduler_warmup.step()
        
        epoch_loss += loss.item()

        # train 클래스별 IoU 계산
        outputs = torch.softmax(outputs, dim=1).cpu()
        outputs = torch.argmax(outputs, dim=1).numpy()

        for class_id in range(N_CLASSES):
            iou = calculate_iou_per_class(np.array(masks.cpu()), np.array(outputs), class_id)
            train_class_ious.append(iou)

    train_class_ious = np.array(train_class_ious).reshape(N_CLASSES, -1)
    train_class_ious = np.mean(train_class_ious, axis=1)
    print("--IoU Scores Train--")
    for class_id, iou in enumerate(train_class_ious):
        print(f'Class{class_id}: {iou:.4f}', end=" ")
        if (class_id+1) % 7 == 0:
            print()

    # mIoU 계산
    train_mIoU = np.mean(train_class_ious)

    # validation
    val_loss = 0
    val_class_ious = []  # 클래스별 IoU를 누적할 리스트 초기화
    with torch.no_grad():
        model.eval()
        for images, masks in tqdm(valid_dataloader):
            images = images.float().to(device)
            masks = masks.long().to(device)
            outputs = model(images)

            # validation loss 계산
            val_loss += criterion(outputs, masks.squeeze(1)).item()

            # validation 클래스별 IoU 계산
            outputs = torch.softmax(outputs, dim=1).cpu()
            outputs = torch.argmax(outputs, dim=1).numpy()

            for class_id in range(N_CLASSES):
                iou = calculate_iou_per_class(np.array(masks.cpu()), np.array(outputs), class_id)
                val_class_ious.append(iou)

    val_class_ious = np.array(val_class_ious).reshape(N_CLASSES, -1)
    val_class_ious = np.mean(val_class_ious, axis=1)
    print("--IoU Scores Valid--")
    for class_id, iou in enumerate(val_class_ious):
        print(f'Class{class_id}: {iou:.4f}', end=" ")
        if (class_id+1) % 7 == 0:
            print("")

    # mIoU 계산
    val_mIoU = np.mean(val_class_ious)

    # 에폭마다 결과 출력 
    print(f"\nEpoch{epoch+1}")
    print(f"Train Loss: {(epoch_loss/len(dataloader))}, Train mIoU Score: {train_mIoU:.4f}")
    print(f"Validation Loss: {val_loss/len(valid_dataloader)}, Validation mIoU Score: {val_mIoU:.4f}")
    print("___________________________________________________________________________________________\n")


  0%|          | 0/35 [00:00<?, ?it/s]

100%|██████████| 35/35 [01:24<00:00,  2.41s/it]


--IoU Scores Train--
Class0: 0.0189 Class1: 0.0181 Class2: 0.0191 Class3: 0.0184 Class4: 0.0182 Class5: 0.0194 Class6: 0.0179 
Class7: 0.0193 Class8: 0.0194 Class9: 0.0187 Class10: 0.0190 Class11: 0.0179 Class12: 0.0186 

100%|██████████| 8/8 [00:19<00:00,  2.48s/it]


--IoU Scores Valid--
Class0: 0.0240 Class1: 0.0126 Class2: 0.0212 Class3: 0.0162 Class4: 0.0260 Class5: 0.0212 Class6: 0.0155 
Class7: 0.0224 Class8: 0.0094 Class9: 0.0258 Class10: 0.0185 Class11: 0.0157 Class12: 0.0254 
Epoch1
Train Loss: 2.67525874546596, Train mIoU Score: 0.0187
Validation Loss: 2.648927390575409, Validation mIoU Score: 0.0195
___________________________________________________________________________________________



...

--IoU Scores Train--
Class0: 0.0191 Class1: 0.0180 Class2: 0.0190 Class3: 0.0189 Class4: 0.0177 Class5: 0.0191 Class6: 0.0182 
Class7: 0.0185 Class8: 0.0193 Class9: 0.0189 Class10: 0.0195 Class11: 0.0180 Class12: 0.0184 

100%|██████████| 8/8 [00:18<00:00,  2.26s/it]


--IoU Scores Valid--
Class0: 0.0232 Class1: 0.0135 Class2: 0.0179 Class3: 0.0162 Class4: 0.0222 Class5: 0.0199 Class6: 0.0163 
Class7: 0.0197 Class8: 0.0109 Class9: 0.0224 Class10: 0.0182 Class11: 0.0147 Class12: 0.0204 
Epoch3
Train Loss: 2.675537313733782, Train mIoU Score: 0.0187
Validation Loss: 2.6990394294261932, Validation mIoU Score: 0.0181
___________________________________________________________________________________________



 46%|████▌     | 16/35 [00:44<00:53,  2.80s/it]


KeyboardInterrupt: 

## Inference

In [ ]:
# test_dataset = CustomDataset(csv_file='./test.csv', transform=transform, infer=True)
# test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)

In [ ]:
# with torch.no_grad():
#     model.eval()
#     result = []
#     for images in tqdm(test_dataloader):
#         images = images.float().to(device)
#         outputs = model(images)
#         outputs = torch.softmax(outputs, dim=1).cpu()
#         outputs = torch.argmax(outputs, dim=1).numpy()
#         # batch에 존재하는 각 이미지에 대해서 반복
#         for pred in outputs:
#             pred = pred.astype(np.uint8)
#             pred = Image.fromarray(pred) # 이미지로 변환
#             pred = pred.resize((960, 540), Image.NEAREST) # 960 x 540 사이즈로 변환
#             pred = np.array(pred) # 다시 수치로 변환
#             # class 0 ~ 11에 해당하는 경우에 마스크 형성 / 12(배경)는 제외하고 진행
#             for class_id in range(12):
#                 class_mask = (pred == class_id).astype(np.uint8)
#                 if np.sum(class_mask) > 0: # 마스크가 존재하는 경우 encode
#                     mask_rle = rle_encode(class_mask)
#                     result.append(mask_rle)
#                 else: # 마스크가 존재하지 않는 경우 -1
#                     result.append(-1)
        

## Submission

In [ ]:
# submit = pd.read_csv('./sample_submission.csv')
# submit['mask_rle'] = result
# submit

In [ ]:
# submit.to_csv('./baseline_submit.csv', index=False)